## EJERCICIO 5

In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [4]:
## Inicializa una sesión en Spark en consola y coloca en una variable llamada playersPersInfo la información
## del archivo t_players_info.csv, t_players_salary.csv, t_players_pers_info.csv considerando la realización
## de un left join; usa como llave primaria el campo Rank y muestra el resultado en pantalla
info_df = spark.read.csv("data/t_players_info.csv", header=True, inferSchema=True)
salary_df = spark.read.csv("data/t_players_salary.csv", sep="|", header=True, inferSchema=True)
pers_df = spark.read.csv("data/t_players_pers_info.csv", inferSchema=True)

pers_renamed_df = pers_df.withColumnRenamed("_c0", "Rank").withColumnRenamed("_c1", "Country").withColumnRenamed(
    "_c2", "Year").withColumnRenamed("_c3", "Date").withColumnRenamed("_c4", "Home")

playersPersInfo = info_df.join(salary_df, ["Rank"], "left").join(pers_renamed_df, ["Rank"], "left")
playersPersInfo.show()

+----+-----------------+----------+--------------------+------+-----------+---------------+------------+------------------+----+----------+--------------------+---+
|Rank|             Name|     Sport|      Wikipedia Page|Gender|  Total Pay|Salary/Winnings|Endorsements|           Country|Year|      Date|                Home|_c5|
+----+-----------------+----------+--------------------+------+-----------+---------------+------------+------------------+----+----------+--------------------+---+
|  55|    Aaron Rodgers|  Football|http://en.wikiped...|  Male|$22,000,000|    $11,000,000| $11,000,000|     United States|1981|26/09/1981|Saginaw, Michigan...|175|
|  55|    Aaron Rodgers|  Football|http://en.wikiped...|  Male|$22,000,000|    $11,000,000| $11,000,000|     United States|1983| 2/12/1983|Chico, California...|188|
|  55|    Aaron Rodgers|  Football|http://en.wikiped...|  Male|$22,000,000|    $14,500,000|  $7,500,000|     United States|1981|26/09/1981|Saginaw, Michigan...|175|
|  55|    

In [5]:
## Muestra el tipo de dato de cada columna
playersPersInfo.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sport: string (nullable = true)
 |-- Wikipedia Page: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Total Pay: string (nullable = true)
 |-- Salary/Winnings: string (nullable = true)
 |-- Endorsements: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Date: string (nullable = true)
 |-- Home: string (nullable = true)
 |-- _c5: integer (nullable = true)



In [6]:
## Filtra solo por los jugadores que no viven en los Estados Unidos
playersNoUSA = playersPersInfo.filter(col("Country") != "United States")
playersNoUSA.show()

+----+---------------+----------+--------------------+------+-----------+---------------+------------+------------------+----+----------+--------------------+---+
|Rank|           Name|     Sport|      Wikipedia Page|Gender|  Total Pay|Salary/Winnings|Endorsements|           Country|Year|      Date|                Home|_c5|
+----+---------------+----------+--------------------+------+-----------+---------------+------------+------------------+----+----------+--------------------+---+
|  95|     Adam Scott|      Golf|https://en.wikipe...|  Male|$17,700,000|     $8,700,000|  $9,000,000|         Australia|1980|16/07/1980| Adelaide, Australia|183|
|  60|Adrian Gonzalez|  Baseball|http://en.wikiped...|  Male|$21,500,000|    $18,000,000|  $3,500,000|Dominican Republic|1982|22/11/1982|San Pedro de Maco...|183|
|  60|Adrian Gonzalez|  Baseball|http://en.wikiped...|  Male|$21,500,000|    $21,100,000|    $400,000|Dominican Republic|1982|22/11/1982|San Pedro de Maco...|183|
|  48| Alex Rodriguez|

In [9]:
## Ordena los jugadores del más joven al más veterano
playersResult = playersNoUSA.sort(desc("Year"))
playersResult.show()

+----+----------------+--------+--------------------+------+-----------+---------------+------------+----------------+----+----------+--------------------+---+
|Rank|            Name|   Sport|      Wikipedia Page|Gender|  Total Pay|Salary/Winnings|Endorsements|         Country|Year|      Date|                Home|_c5|
+----+----------------+--------+--------------------+------+-----------+---------------+------------+----------------+----+----------+--------------------+---+
|  16|          Neymar|  Soccer|http://en.wikiped...|  Male|$33,600,000|    $17,600,000| $16,000,000|          Brazil|1992|05/02/1992|Mogi das Cruzes, ...|175|
|  66|     Eli Manning|Football|http://en.wikiped...|  Male|$21,000,000|    $10,000,000| $11,000,000|          Mexico|1990|18/07/1990|Guadalajara, Jali...|180|
|  66|      Drew Brees|Football|http://en.wikiped...|  Male|$21,000,000|    $10,000,000| $11,000,000|          Mexico|1990|18/07/1990|Guadalajara, Jali...|180|
|  66|     Eli Manning|Football|http://e

In [11]:
## Escribe el resultado en un nuevo CSV delimitado por pipes "|" y nómbralo csv_result_players_info.csv
playersResult.write.mode("overwrite").csv("data/csv_result_players_info.csv", header=True, sep="|")